In [1]:
#! /usr/bin/env python
"""
Compute elevation statistics for the debris-covered areas in each latitude and longitude
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from scipy import ndimage
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib
# from imview.lib import pltlib


import globaldebris_input as input


# csv_ending = '_mb_bins_wdc_emvel_offset.csv'
# verbose = False
# close_fig = True
# extra_layers = True

In [2]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(input.debriscover_fp + input.debriscover_fn_dict[input.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

print('All DC glaciers:', dc_shp.shape[0], 'All DC Area (km2):', dc_shp.DC_Area_v2.sum() / 1e6)

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > input.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > input.dc_area_threshold))
                        & (dc_shp['Area'] > input.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)
dc_shp_subset['CenLon_360'] = dc_shp_subset['CenLon']
dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'] = 360 + dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360']

print('Subset DC glaciers:', dc_shp_subset.shape[0], 'Subset DC Area (km2):', dc_shp_subset.DC_Area_v2.sum() / 1e6)

# print(dc_shp_subset.RGIId.values)
dc_shp_subset

# # # ===== LOAD RGI DATA (works for multiple regions together) =====
# # rgi_fns = []
# # for i in os.listdir(input.rgi_fp):
# # #     print(i)
# #     for reg_no in input.roi_rgidict[input.roi]:
# #         reg_str = str(reg_no).zfill(2)
# #         if i.startswith(reg_str) and i.endswith('.csv'):
# #             rgi_fns.append(i)
# # rgi_fns = sorted(rgi_fns)

# # # Load RGI data
# # rgi_data = None
# # for i in rgi_fns:
# #     rgi_reg = pd.read_csv(input.rgi_fp + i)
    
# #     if rgi_data is None:
# #         rgi_data = rgi_reg
# #     else:
# #         rgi_data = pd.concat((rgi_data, rgi_reg), axis=0)
# # rgi_data.reset_index(inplace=True, drop=True)
# # rgi_data

All DC glaciers: 1011 All DC Area (km2): 174.176324
Subset DC glaciers: 43 Subset DC Area (km2): 92.158983


RGIId         GLIMSId   BgnDate   EndDate      CenLon     CenLat  \
0   RGI60-18.00171  G168039E44594S  19789999  -9999999  168.037603 -44.591427   
1   RGI60-18.00179  G168020E44606S  19789999  -9999999  168.018668 -44.605576   
2   RGI60-18.00569  G168242E44591S  19789999  -9999999  168.235877 -44.590591   
3   RGI60-18.00631  G168624E44538S  19789999  -9999999  168.620338 -44.539415   
4   RGI60-18.00661  G168556E44463S  19789999  -9999999  168.553044 -44.465521   
5   RGI60-18.00686  G168609E44455S  19789999  -9999999  168.602431 -44.456298   
6   RGI60-18.00707  G168578E44439S  19789999  -9999999  168.578927 -44.439472   
7   RGI60-18.00716  G168482E44469S  19789999  -9999999  168.481966 -44.469631   
8   RGI60-18.00718  G168495E44485S  19789999  -9999999  168.487470 -44.487016   
9   RGI60-18.00815  G168358E44502S  19789999  -9999999  168.355187 -44.499472   
10  RGI60-18.00853  G168363E44474S  19789999  -9999999  168.362613 -44.475891   
11  RGI60-18.00891  G168590E44384S  19789999  -9999999  168.593038 -44.384984   
12  RGI60-18.00947  G168764E44358S  19789999  -9999999  168.768386 -44.357248   
13  RGI60-18.00996  G168818E44336S  19789999  -9999999  168.818171 -44.326711   
14  RGI60-18.01018  G168768E44396S  19789999  -9999999  168.768167 -44.395203   
15  RGI60-18.01130  G169437E44072S  19789999  -9999999  169.438836 -44.070276   
16  RGI60-18.01307  G168868E44222S  19789999  -9999999  168.867654 -44.218888   
17  RGI60-18.01852  G170009E43687S  19789999  -9999999  170.009573 -43.687832   
18  RGI60-18.01889  G169988E43731S  19789999  -9999999  169.994145 -43.731389   
19  RGI60-18.01958  G169947E43804S  19789999  -9999999  169.946781 -43.803873   
20  RGI60-18.01959  G170034E43739S  19789999  -9999999  170.038546 -43.716271   
21  RGI60-18.02210  G170327E43453S  19789999  -9999999  170.330915 -43.453851   
22  RGI60-18.02230  G170115E43556S  19789999  -9999999  170.120018 -43.556089   
23  RGI60-18.02257  G170068E43617S  19789999  -9999999  170.070270 -43.616274   
24  RGI60-18.02270  G170165E43501S  19789999  -9999999  170.168032 -43.500657   
25  RGI60-18.02276  G170292E43590S  19789999  -9999999  170.289192 -43.591231   
26  RGI60-18.02298  G170521E43401S  19789999  -9999999  170.516036 -43.399023   
27  RGI60-18.02342  G170238E43565S  19789999  -9999999  170.238053 -43.559504   
28  RGI60-18.02436  G170118E43618S  19789999  -9999999  170.119435 -43.610969   
29  RGI60-18.02450  G170292E43488S  19789999  -9999999  170.292035 -43.489276   
30  RGI60-18.02472  G170362E43486S  19789999  -9999999  170.362151 -43.490760   
31  RGI60-18.02499  G170492E43447S  19789999  -9999999  170.494725 -43.444484   
32  RGI60-18.02502  G170080E43581S  19789999  -9999999  170.098251 -43.578289   
33  RGI60-18.02504  G170379E43558S  19789999  -9999999  170.374646 -43.538126   
34  RGI60-18.02505  G170449E43476S  19789999  -9999999  170.432844 -43.485789   
35  RGI60-18.02508  G170591E43444S  19789999  -9999999  170.562472 -43.433323   
36  RGI60-18.03066  G170843E43306S  19789999  -9999999  170.844198 -43.295033   
37  RGI60-18.03078  G170706E43335S  19789999  -9999999  170.706233 -43.334050   
38  RGI60-18.03156  G170913E43237S  19789999  -9999999  170.908916 -43.242115   
39  RGI60-18.03167  G170637E43308S  19789999  -9999999  170.638651 -43.304808   
40  RGI60-18.03181  G170734E43324S  19789999  -9999999  170.735869 -43.322688   
41  RGI60-18.03191  G170925E43205S  19789999  -9999999  170.918699 -43.200712   
42  RGI60-18.03209  G170940E43193S  19789999  -9999999  170.938737 -43.192698   

   O1Region O2Region    Area  Zmin  ...   DC_Area  DC_BgnDate  DC_EndDate  \
0        18        1   2.790  1160  ...    810000        2013        2017   
1        18        1   2.189   850  ...    240300        2013        2017   
2        18        1   2.559  1503  ...    373500        2013        2017   
3        18        1   2.472  1465  ...    445500        2013        2017   
4        18        1   7.027  1073  ...   1637100        2013  

In [3]:
# ===== SELECT GLACIERS WITH DEBRIS ====
ds = xr.open_dataset(input.metdata_fp + '../' + input.metdata_elev_fn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(dc_shp_subset['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(dc_shp_subset['CenLon_360'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))

latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))
dc_shp_subset['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
dc_shp_subset['latlon_unique_no'] = dc_shp_subset['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(dc_shp_subset['latlon_unique_no'])), '\n\n')
# print(dc_shp_subset_wdata.loc[0:5,['RGIId', 'CenLat', 'CenLon', 'larsen_fn', 'braun_fn', 'latlon_unique_no']])

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# print(latlon_list)

# Pickle unique lat/lons that will be used for melt model
with open(input.latlon_unique_fp + input.latlon_unique_dict[input.roi], 'wb') as f:
    pickle.dump(latlon_list, f)

unique lat/lons: 12 




In [4]:
# ===== LOAD GLACIERS WITH DATA =====
dc_shp_subset['mb_fullfn'] = np.nan
mb_binned_fp = input.mb_binned_fp

mb_fns = []
mb_rgiids = []
for i in os.listdir(mb_binned_fp):
    if i.endswith('_mb_bins.csv'):
        mb_fns.append(mb_binned_fp + i)
        rgiid_raw = i.split('_')[0]
        rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
        mb_rgiids.append(rgiid)
mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
mb_fn_df['RGIId'] = mb_rgiids
mb_fn_df['mb_fullfn'] = mb_fns

# Find glaciers that are debris-covered
mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) if value in list(dc_shp_subset.RGIId.values)]
mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')

print('Debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')

mb_fn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fullfn'].values))

dc_shp_subset['mb_fullfn'] = dc_shp_subset.RGIId.map(mb_fn_dict)

Debris-covered glaciers: 43 




In [5]:
# ===== SELECT GLACIERS WITH DATA ====
dc_shp_subset_wdata = dc_shp_subset.dropna(subset=['mb_fullfn']).copy()
dc_shp_subset_wdata.reset_index(inplace=True, drop=True)
ds = xr.open_dataset(input.metdata_fp + '../' + input.metdata_elev_fn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(dc_shp_subset_wdata['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(dc_shp_subset_wdata['CenLon_360'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))

latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))
dc_shp_subset_wdata['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
dc_shp_subset_wdata['latlon_unique_no'] = dc_shp_subset_wdata['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(dc_shp_subset_wdata['latlon_unique_no'])), '\n\n')
# print(dc_shp_subset_wdata.loc[0:5,['RGIId', 'CenLat', 'CenLon', 'larsen_fn', 'braun_fn', 'latlon_unique_no']])

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# print(latlon_list)

# Pickle unique lat/lons that will be used for melt model
with open(input.latlon_unique_fp + input.latlon_unique_dict[input.roi], 'wb') as f:
    pickle.dump(latlon_list, f)

unique lat/lons: 12 




In [6]:
# ===== Load Glaciers =====
rgiid_list = [x.split('-')[1] for x in dc_shp_subset_wdata['RGIId'].values]
main_glac_rgi = input.selectglaciersrgitable(rgiid_list)
# add filenames
main_glac_rgi['mb_fn'] = np.nan
main_glac_rgi['mb_fn'] = main_glac_rgi.RGIId.map(mb_fn_dict)

43 glaciers in region 18 are included in this model run: ['00171', '00179', '00569', '00631', '00661', '00686', '00707', '00716', '00718', '00815', '00853', '00891', '00947', '00996', '01018', '01130', '01307', '01852', '01889', '01958', '01959', '02210', '02230', '02257', '02270', '02276', '02298', '02342', '02436', '02450', '02472', '02499', '02502', '02504', '02505', '02508', '03066', '03078', '03156', '03167', '03181', '03191', '03209'] and more
This study is focusing on 43 glaciers in region [18]


In [20]:
# ===== DEBRIS ELEVATION STATS ====================================================================================
# Glaciers with data
glac_wobs_fns = []
rgiid_wobs = []
for i in os.listdir(input.mb_binned_fp):
#     if i.endswith('_mb_bins_wdc_emvel_offset.csv'):
    if i.endswith('_mb_bins.csv'):
        rgiid_reg = int(i.split('.')[0])
        if int(rgiid_reg) in input.roi_rgidict[input.roi]:
            glac_wobs_fns.append(i)
            if rgiid_reg < 10:
                rgiid_wobs.append(i[0:7])
            else:
                rgiid_wobs.append(i[0:8])
        
glac_wobs_fns = sorted(glac_wobs_fns)
rgiid_wobs = sorted(rgiid_wobs)

# ===== SELECT GLACIERS WITH DATA =====
main_glac_rgi_wobs = input.selectglaciersrgitable(rgiid_wobs)
main_glac_rgi_wobs['mb_bin_fn'] = glac_wobs_fns 
main_glac_rgi_wobs['CenLon_360'] = main_glac_rgi_wobs['CenLon']
main_glac_rgi_wobs.loc[main_glac_rgi_wobs['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi_wobs.loc[main_glac_rgi_wobs['CenLon_360'] < 0, 'CenLon_360'])
ds = xr.open_dataset(input.metdata_fp + '../' + input.metdata_elev_fn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(main_glac_rgi_wobs['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi_wobs['CenLon_360'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))
latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))
main_glac_rgi_wobs['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_wobs['latlon_unique_no'] = main_glac_rgi_wobs['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_wobs['latlon_unique_no'])), '\n\n')
# print(dc_shp_subset_wdata.loc[0:5,['RGIId', 'CenLat', 'CenLon', 'larsen_fn', 'braun_fn', 'latlon_unique_no']])

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# ===== CALCULATE DEBRIS ELEVATION STATS FOR GLACIERS WITH DATA FOR EACH UNIQUE LAT/LON ======
elev_stats_latlon_dict = {}
latlon_list_updated = []
rgiid_4cal = []
for nlatlon, latlon_unique in enumerate(np.unique(main_glac_rgi_wobs.latlon_unique_no)):
# for nlatlon, latlon_unique in enumerate([np.unique(main_glac_rgi_wobs.latlon_unique_no)[3]]):

    main_glac_rgi_subset = main_glac_rgi_wobs[main_glac_rgi_wobs['latlon_unique_no'] == latlon_unique]

    # Debris elevation stats should be done by lat/lon
    df_all = None
    elev_list_all = []
    df_idx_count = 0
    count_width_passes = 0
    for nglac, glac_fn in enumerate(main_glac_rgi_subset.mb_bin_fn.values):
#     for nglac, glac_fn in enumerate([main_glac_rgi_subset.mb_bin_fn.values[3]]):
        
        print(nlatlon, nglac, glac_fn.split('_')[0])

        df_raw = pd.read_csv(input.mb_binned_fp + glac_fn)
        df = df_raw.dropna(subset=['mb_bin_mean_mwea'])
        df_debris = df[(df['vm_med'] < input.vel_threshold) & (df['dc_bin_area_perc'] > input.debrisperc_threshold)
                       & (df['dc_bin_count_valid'] > 0)]

        df_idx = df_debris.index.values
        df_idx_count += len(df_idx)
        
        # Widths
        rgiid = 'RGI60-' + glac_fn.split('_')[0]
        widths_fp = input.oggm_fp + 'widths/' + 'RGI60-' + rgiid.split('-')[1].split('.')[0] + '/'
        widths_fn = rgiid + '_widths_m.csv'
        try:
            widths_df = pd.read_csv(widths_fp + widths_fn)
            h = widths_df['elev'].values
            widths_m = widths_df['width_m'].values
        except:
            widths_df = None

        if len(df_idx) > 0 and widths_df is not None:
            # only work with terminus
            df_idx_dif = list(df_idx[1:] - df_idx[:-1])
            if np.sum(df_idx_dif) == len(df_idx)-1:
                df_idx_nojump = df_idx
            else:
                idx_jumpinbins = df_idx_dif.index(next(filter(lambda x: x>1, df_idx_dif)))
                df_idx_nojump = df_idx[0:idx_jumpinbins+1]

            df_debris_nojump = df_debris.loc[df_idx_nojump,:]
            df_debris_nojump.reset_index(inplace=True, drop=True)
            
            # Median width to ensure terminus velocities can be estimated
            width_median = np.median(widths_m[np.where(h < df_debris_nojump['bin_center_elev_m'].max())[0]])
            
            if width_median > input.width_min_dict[input.roi]:
                for nelev, elev in enumerate(list(df_debris_nojump['bin_center_elev_m'].values)):
                    elev_list_single = list(np.repeat(elev, df_debris_nojump.loc[nelev,'dc_bin_count_valid']))
                    elev_list_all.extend(elev_list_single)
                count_width_passes += 1
                
            rgiid_4cal.append(glac_fn.split('_')[0])
        
    if df_idx_count > 0 and count_width_passes > 0:
        dc_zmean = np.mean(elev_list_all)
        dc_zstd = np.std(elev_list_all)
        dc_zmed = malib.fast_median(elev_list_all)
        dc_zmad = malib.mad(elev_list_all)

        lat_deg = float(ds.latitude[latlon_unique_dict_reversed[latlon_unique][0]].values)
        lon_deg = float(ds.longitude[latlon_unique_dict_reversed[latlon_unique][1]].values)
        elev_stats_latlon_dict[lat_deg,lon_deg] = [dc_zmean, dc_zstd, dc_zmed, dc_zmad]
        latlon_list_updated.append((lat_deg, lon_deg))
        
print('unique lat/lons updated:', len(latlon_list_updated))
# Update pickle of unique lat/lons that will be used for melt model
with open(input.latlon_unique_fp + input.latlon_unique_dict[input.roi], 'wb') as f:
    pickle.dump(latlon_list_updated, f)

43 glaciers in region 18 are included in this model run: ['00171', '00179', '00569', '00631', '00661', '00686', '00707', '00716', '00718', '00815', '00853', '00891', '00947', '00996', '01018', '01130', '01307', '01852', '01889', '01958', '01959', '02210', '02230', '02257', '02270', '02276', '02298', '02342', '02436', '02450', '02472', '02499', '02502', '02504', '02505', '02508', '03066', '03078', '03156', '03167', '03181', '03191', '03209'] and more
This study is focusing on 43 glaciers in region [18]
unique lat/lons: 12 


0 0 18.03066
0 1 18.03078
0 2 18.03167
0 3 18.03181
1 0 18.03156
1 1 18.03191
1 2 18.03209
2 0 18.02230
2 1 18.02257
2 2 18.02436
2 3 18.02502
3 0 18.02210
3 1 18.02270
3 2 18.02276
3 3 18.02342
3 4 18.02450
3 5 18.02472
4 0 18.02298
4 1 18.02499
4 2 18.02504
4 3 18.02505
4 4 18.02508
5 0 18.01852
5 1 18.01889
5 2 18.01958
5 3 18.01959
6 0 18.01130
7 0 18.00947
7 1 18.00996
7 2 18.01307
8 0 18.00171
8 1 18.00179
9 0 18.00569
9 1 18.00815
9 2 18.00853
10 0 18.00631
1

In [21]:
# Statistics of data coverage
rgiid_4cal = sorted(rgiid_4cal)
main_glac_rgi_4cal = input.selectglaciersrgitable(rgiid_4cal)
dc_area_dict = dict(zip(dc_shp.RGIId.values, dc_shp.DC_Area_v2.values))
main_glac_rgi_4cal['DC_Area_v2'] = main_glac_rgi_4cal.RGIId.map(dc_area_dict)
print('\nDC glaciers (used for cal):', main_glac_rgi_4cal.shape[0], 
      'DC Area (used for cal, km2):', main_glac_rgi_4cal.DC_Area_v2.sum() / 1e6)

42 glaciers in region 18 are included in this model run: ['00171', '00179', '00569', '00631', '00661', '00686', '00707', '00716', '00718', '00815', '00853', '00891', '00947', '00996', '01018', '01130', '01307', '01852', '01889', '01958', '01959', '02210', '02230', '02257', '02270', '02276', '02298', '02342', '02436', '02472', '02499', '02502', '02504', '02505', '02508', '03066', '03078', '03156', '03167', '03181', '03191', '03209'] and more
This study is focusing on 42 glaciers in region [18]

DC glaciers (used for cal): 42 DC Area (used for cal, km2): 91.145419


In [22]:
# ===== ADD DEBRIS ELEVATION STATS TO MET DATA ======
overwrite_dc_stats = True
for nlatlon, latlon in enumerate(latlon_list_updated):
# for nlatlon, latlon in enumerate([latlon_list_updated[0]]):
    
    lat_deg = latlon[0]
    lon_deg = latlon[1]
    
    print(lat_deg, lon_deg)
    
    if lat_deg < 0:
        lat_str = 'S-'
    else:
        lat_str = 'N-' 

    # ===== Meteorological data =====
    metdata_fn = input.metdata_fn_sample.replace('XXXX', 
                                                 str(int(np.abs(lat_deg)*100)) + lat_str + str(int(lon_deg*100)) + 'E-')
    
    ds = xr.open_dataset(input.metdata_fp + metdata_fn)    
#     print('  ', ds.dc_zmean.values, elev_stats_latlon_dict[latlon][0])
    if 'dc_zmean' not in list(ds.keys()) or overwrite_dc_stats:
        # Add stats
        ds['dc_zmean'] = elev_stats_latlon_dict[latlon][0]
        ds['dc_zmean'].attrs = {'units':'m a.s.l.', 'long_name':'Mean debris cover elevation', 'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zstd'] = elev_stats_latlon_dict[latlon][1]
        ds['dc_zstd'].attrs = {'units':'m a.s.l.', 'long_name':'Standard deviation of debris cover elevation', 'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmed'] = elev_stats_latlon_dict[latlon][2]
        ds['dc_zmed'].attrs = {'units':'m a.s.l.', 'long_name':'Median debris cover elevation', 'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmad'] = elev_stats_latlon_dict[latlon][3]
        ds['dc_zmad'].attrs = {'units':'m a.s.l.', 'long_name':'Median absolute deviation of debris cover elevation', 'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}

        try:
            ds.close()
        except:
            continue
            
        # Export updated dataset
#         ds.to_netcdf(input.metdata_fp + '../HMA_modified/' + metdata_fn)
        ds.to_netcdf(input.metdata_fp + metdata_fn, mode='a')
    else:
        print(lat_deg, lon_deg, 'exists')

-43.25 170.75
-43.25 171.0
-43.5 170.0
-43.5 170.25
-43.5 170.5
-43.75 170.0
-44.0 169.5
-44.25 168.75
-44.5 168.0
-44.5 168.25
-44.5 168.5
-44.5 168.75


In [7]:
# ==== OLD FILE OF LOADIING MULTIPLE DATASETS =====
# # ===== LOAD GLACIERS WITH LARSEN DATA =====
# dc_shp_subset['larsen_fullfn'] = np.nan
# larsen_fullfn_dict = {}
# if 'larsen' in input.mb_datasets:
#     mb_summary = pd.read_csv(input.larsen_fp + input.larsen_fn)
    
#     # Find glaciers that are debris-covered
#     larsen_dc_rgiid = [value for value in list(mb_summary.RGIId.values) 
#                        if value in list(dc_shp_subset.RGIId.values)]

#     mb_summary_dc = mb_summary[mb_summary['RGIId'].isin(larsen_dc_rgiid)]
#     mb_summary_dc = mb_summary_dc.sort_values('RGIId')
#     mb_summary_dc.reset_index(inplace=True, drop=True)
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Maclaren', 'name'] = 'MacLaren'
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Tlikakila Fork', 'name'] = 'TlikakilaGlacierFork'
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Tlikakila N. Fork', 'name'] = 'TlikakilaNorthFork'
#     mb_summary_dc['larsen_fullfn'] = np.nan
    
#     for n, glac_name in enumerate(mb_summary_dc.name.values):
# #     for n, glac_name in enumerate([mb_summary_dc.name.values[47]]):
# #         print(n, glac_name)
            
#         glac_name = glac_name.replace(' ', '')
#         glac_fns = []
#         start_yr = []
#         end_yr = []
#         for i in os.listdir(input.larsen_binned_fp):
#             if i.startswith(glac_name):
#                 glac_fns.append(i)
#                 start_yr.append(i.split('.')[1][0:4])
#                 end_yr.append(i.split('.')[2][0:4])
                
#         if len(glac_fns) > 0:
#             yr_dif = np.array(end_yr).astype(int) - np.array(start_yr).astype(int)
#             mb_fn = glac_fns[np.where(yr_dif == yr_dif.max())[0][0]]
            
#             # ===== Process Larsen dataset =====
#             larsen_data_raw = np.genfromtxt(input.larsen_binned_fp + mb_fn, skip_header=3)
#             larsen_data_header = ['E', 'DZ', 'DZ25', 'DZ75', 'AAD', 'MassChange', 'MassBal', 'NumData']
#             larsen_data = pd.DataFrame(larsen_data_raw, columns=larsen_data_header)
#             larsen_data['std from DZ25'] = np.absolute(larsen_data['DZ'] - larsen_data['DZ25']) / 0.67
#             larsen_data['std from DZ75'] = np.absolute(larsen_data['DZ'] - larsen_data['DZ75']) / 0.67
#             larsen_data[' dhdt_bin_std_ma'] = (larsen_data['std from DZ25'] + larsen_data['std from DZ75']) / 2
#             larsen_data[' mb_bin_std_mwea'] = larsen_data[' dhdt_bin_std_ma'] * 900 / 1000
#             larsen_data['AAD'] = larsen_data['AAD'] / 1e6
#             larsen_data['startyear'] = int(mb_fn.split('.')[1][0:4])
#             larsen_data['endyear'] = int(mb_fn.split('.')[2][0:4])
#             larsen_data = larsen_data.rename({'E': '# bin_center_elev_m',
#                                               'DZ': ' dhdt_bin_mean_ma',
#                                               'MassBal': ' mb_bin_mean_mwea',
#                                               'AAD': ' z1_bin_area_valid_km2',
#                                              }, axis='columns')
#             new_fn = mb_summary_dc.loc[n,'RGIId'].split('-')[1][1:] + '_larsen_mb_bins.csv'
#             larsen_data.to_csv(input.larsen_binned_fp + new_fn, index=False)
            
#             mb_summary_dc.loc[n, 'larsen_fullfn'] = input.larsen_binned_fp + new_fn
            
#         else:
#             print(n, glac_name, 'has no file\n')

#     mb_summary_dc.dropna(subset=['larsen_fullfn'], inplace=True)
#     mb_summary_dc.reset_index(inplace=True, drop=True)
    
#     print('Larsen debris-covered glaciers:', mb_summary_dc.shape[0], '\n\n')
    
#     larsen_fullfn_dict = dict(zip(mb_summary_dc['RGIId'].values, mb_summary_dc['larsen_fullfn'].values))
# #     print(larsen_fullfn_dict)
#     dc_shp_subset['larsen_fullfn'] = dc_shp_subset.RGIId.map(larsen_fullfn_dict)

# # ===== LOAD GLACIERS WITH BRAUN DATA =====
# dc_shp_subset['braun_fullfn'] = np.nan
# braun_fullfn_dict = {}
# if 'braun' in input.mb_datasets:
#     mb_binned_fp = input.main_directory + '/../mb_data/Braun/binned_data/'
# #     mb_binned_fp = input.mb_binned_fp
    
#     mb_fns = []
#     braun_rgiids = []
#     for i in os.listdir(mb_binned_fp):
#         if i.endswith('_mb_bins.csv'):
#             mb_fns.append(mb_binned_fp + i)
#             rgiid_raw = i.split('_')[0]
#             rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
#             braun_rgiids.append(rgiid)
#     braun_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'braun_fn'])
#     braun_fn_df['RGIId'] = braun_rgiids
#     braun_fn_df['braun_fullfn'] = mb_fns
    
#     # Find glaciers that are debris-covered
#     braun_dc_rgiid = [value for value in list(braun_fn_df.RGIId.values) 
#                        if value in list(dc_shp_subset.RGIId.values)]
#     braun_fn_df_dc = braun_fn_df[braun_fn_df['RGIId'].isin(braun_dc_rgiid)]
#     braun_fn_df_dc = braun_fn_df_dc.sort_values('RGIId')
    
#     print('Braun debris-covered glaciers:', braun_fn_df_dc.shape[0], '\n\n')
    
#     braun_fullfn_dict = dict(zip(braun_fn_df_dc['RGIId'].values, braun_fn_df_dc['braun_fullfn'].values))
    
#     dc_shp_subset['braun_fullfn'] = dc_shp_subset.RGIId.map(braun_fullfn_dict)

# # ===== LOAD GLACIERS WITH SHEAN DATA =====
# dc_shp_subset['shean_fullfn'] = np.nan
# shean_fullfn_dict = {}
# if 'shean' in input.mb_datasets:
# #     mb_binned_fp = input.main_directory + '/../mb_data/Shean_2019_0213/mb_combined_20190213_nmad_bins/'
#     mb_binned_fp = input.mb_binned_fp
    
#     mb_fns = []
#     rgiids = []
#     for i in os.listdir(mb_binned_fp):
#         if i.endswith('_mb_bins.csv'):
#             mb_fns.append(mb_binned_fp + i)
#             rgiid_raw = i.split('_')[0]
#             rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
#             rgiids.append(rgiid)
#     mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
#     mb_fn_df['RGIId'] = rgiids
#     mb_fn_df['mb_fullfn'] = mb_fns
    
#     # Find glaciers that are debris-covered
#     mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) 
#                    if value in list(dc_shp_subset.RGIId.values)]
#     mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
#     mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')
    
#     print('shean debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')
    
#     shean_fullfn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fullfn'].values))
# #     print(shea_fullfn_dict)
#     dc_shp_subset['shean_fullfn'] = dc_shp_subset.RGIId.map(shean_fullfn_dict)

# # Merge dictionaries together
# mb_fn_dict = dict(list(larsen_fullfn_dict.items()) + list(braun_fullfn_dict.items()) + 
#                   list(shean_fullfn_dict.items()))